## University of Vienna — Winter Semester 2022
# 250085 VO Tensor methods for data science and scientific computing
## Ass.-Prof. Dr. Vladimir Kazeev
# Homework Assignment 1

In [63]:
using LinearAlgebra, Printf, Plots, LaTeXStrings, Test

nothing;

In [64]:
n = 901

t = collect(0:n-1)./(n-1)
t = 2 .*t .-1

f(x,y) = 1/(1+x^2+y^2)

g(x,y) = sqrt(x^2+y^2)*(1+cos(15*x+22*y)/2)

A = [ f(t[i],t[j]) for i ∈ 1:n, j ∈ 1:n ]
B = [ g(t[i],t[j]) for i ∈ 1:n, j ∈ 1:n ]

nothing;

In [65]:
p = plot(; label="", layout=2, size=(900,400))

contour!(p[1], t, t, A, linewidth=1.0, title="f")
contour!(p[2], t, t, B, linewidth=1.0, title="g")

In [66]:
function iluu(A::AbstractMatrix, r::Int)
    # L,U,k = ilu!(A, r)
    #     attempts to compute an r-step incomplete LU decomposition
    #       A = L*U
    #       of the given matrix A of size m x n
    #     returns a k-step incomplete LU decomposition of A
    #     with the maximal possible number k ∈ 0:r of steps
    #
    if length(A) == 0
        # the matrix has a zero number of rows or a zero number of columns (or both)
        throw(ArgumentError("the matrix is empty"))
    end
        
    m,n = size(A)
    if r ∉ 1:min(m, n)
        throw(ArgumentError("1 ≦ r ≦ m,n is required"))
    end
    U = A # by the way, the matrix is not copied here
          # variables A and U refer to the same contiguous block of memory
    L = Matrix{eltype(A)}(I, m, m) # initialize L as the identity matrix of order m and the same type as A
    if m == 1
        # nothing is to be done in this case
        # we only need to decide whether what we have is a one-step incomplete LU decomposition or not
        # by looking at U
        # in order to return k=1 or k=1 respectively
        if U[1,1] == 0
            k = 0
        else
            k = 1
        end
        return L,U,k
    end
    # perform Gaussian elimination
    for k ∈ 1:r
        if U[k,k] == 0
            # we are done with elimination: step k cannot be performed
            return L,U,k-1
        end
        # perform step k
        # compute Gaussian-elimination coefficients
        L[k+1:m,k] = U[k+1:m,k] ./ U[k,k]
        # update the matrix in rows k+1 to m and columns k+1 to n
        U[k+1:m,k:k] .= 0 # we can explicitly eliminate the corresponding entries in U
        U[k+1:m,k+1:n] = U[k+1:m,k+1:n] .- L[k+1:m,k:k]*U[k:k,k+1:n] # compute the Schur complement
    end
    k = r
    return L,U,k
end

iluu (generic function with 1 method)

In [67]:
function pivot_complete(S::AbstractMatrix)
    m,n = size(S)
    _,ind = findmax(abs.(S)) # find the index of an entry of S
                             # that is maximal in absolute value
                             # ind is some strange object of type CartesianIndex
    i = ind[1]; j=ind[2]     # so we convert it into two indices, taking itno account the offset
                             # now S[i,j] has the maximum absolute value among the entries of S
    return i,j
end

function pilu!(A::AbstractMatrix, r::Int, pivot)
    # L,U,P,Q,k = pilu(A, r)
    #     attempts to compute a pivoted r-step incomplete LU decomposition
    #       P A Q' = L*U
    #       of the given matrix A of size m x n
    #     returns a k-step pivoted incomplete LU decomposition of A
    #     with the maximal possible number k ∈ 0:r of steps
    # We are not concerned with efficiency at this point,
    #                            so L,P,Q are stored as full matrices
    #
    if length(A) == 0
        # the matrix has a zero number of rows or a zero number of columns (or both)
        throw(ArgumentError("the matrix is empty"))
    end
        
    m,n = size(A)
    if r ∉ 1:min(m, n)
        throw(ArgumentError("1 ≦ r ≦ m,n is required"))
    end
    U = A # by the way, the matrix is not copied here
          # variables A and U refer to the same contiguous block of memory
    # initialize L,P,Q as identity matrices
    L = Matrix{eltype(A)}(I, m, m)
    P = Matrix{Int}(I, m, m)
    Q = Matrix{Int}(I, n, n)
    # perform Gaussian elimination
    for k ∈ 1:r
        S = U[k:m,k:n]
        i,j = pivot(S)
        if S[i,j] == 0
            # the pivoting function pointed us to a zero entry,
            # which cannot be used as a pivot,
            # so step k cannot be performed with this pivot;
            # we are done with elimination
            return L,U,P,Q,k-1
        end
        # i and j indicate an entry of S, so we need to shift theese indices
        # to obtain the same entry in U
        i = k-1 + i
        j = k-1 + j
        # perform step k using entry (i,j) as pivot
        # apply row and column exchange to L,U,P,Q
        L[[k, i],1:k-1] = L[[i, k],1:k-1]
        U[[k, i],:] = U[[i, k],:]
        U[:,[k, j]] = U[:,[j, k]]
        P[[k, i],:] = P[[i, k],:]
        Q[[k, j],:] = Q[[j, k],:]
        # compute Gaussian-elimination coefficients
        L[k+1:m,k] = U[k+1:m,k] ./ U[k,k]
        # update the matrix in rows k+1 to m and columns k+1 to n
        U[k+1:m,k:k] .= 0 # we can explicitly eliminate the corresponding entries in U
        U[k+1:m,k+1:n] = U[k+1:m,k+1:n] .- L[k+1:m,k:k]*U[k:k,k+1:n] # compute the Schur complement
    end
    k = r
    return L,U,P,Q,k
end

nothing;

In [68]:
A

901×901 Matrix{Float64}:
 0.333333  0.333827  0.334322  0.334816  …  0.334322  0.333827  0.333333
 0.333827  0.334323  0.334819  0.335315     0.334819  0.334323  0.333827
 0.334322  0.334819  0.335316  0.335814     0.335316  0.334819  0.334322
 0.334816  0.335315  0.335814  0.336313     0.335814  0.335315  0.334816
 0.335312  0.335811  0.336312  0.336812     0.336312  0.335811  0.335312
 0.335807  0.336308  0.33681   0.337312  …  0.33681   0.336308  0.335807
 0.336303  0.336806  0.337309  0.337812     0.337309  0.336806  0.336303
 0.336799  0.337303  0.337808  0.338313     0.337808  0.337303  0.336799
 0.337295  0.337801  0.338307  0.338814     0.338307  0.337801  0.337295
 0.337792  0.3383    0.338807  0.339315     0.338807  0.3383    0.337792
 0.338289  0.338798  0.339307  0.339817  …  0.339307  0.338798  0.338289
 0.338787  0.339297  0.339808  0.340319     0.339808  0.339297  0.338787
 0.339285  0.339796  0.340309  0.340821     0.340309  0.339796  0.339285
 ⋮                        

In [69]:

L,U,k=iluu(copy(A),100)

([1.0 0.0 … 0.0 0.0; 1.001482028549244 1.0 … 0.0 0.0; … ; 1.001482028549244 0.9999999999244036 … 1.0 0.0; 1.0 0.0 … 0.0 1.0], [0.3333333333333333 0.333827342849748 … 0.333827342849748 0.3333333333333333; 0.0 7.343095214107365e-7 … 7.343095213552253e-7 0.0; … ; 0.0 0.0 … -2.043320586997022e-16 2.8483638555187947e-17; 0.0 0.0 … 0.0 0.0], 100)

In [70]:
A

901×901 Matrix{Float64}:
 0.333333  0.333827  0.334322  0.334816  …  0.334322  0.333827  0.333333
 0.333827  0.334323  0.334819  0.335315     0.334819  0.334323  0.333827
 0.334322  0.334819  0.335316  0.335814     0.335316  0.334819  0.334322
 0.334816  0.335315  0.335814  0.336313     0.335814  0.335315  0.334816
 0.335312  0.335811  0.336312  0.336812     0.336312  0.335811  0.335312
 0.335807  0.336308  0.33681   0.337312  …  0.33681   0.336308  0.335807
 0.336303  0.336806  0.337309  0.337812     0.337309  0.336806  0.336303
 0.336799  0.337303  0.337808  0.338313     0.337808  0.337303  0.336799
 0.337295  0.337801  0.338307  0.338814     0.338307  0.337801  0.337295
 0.337792  0.3383    0.338807  0.339315     0.338807  0.3383    0.337792
 0.338289  0.338798  0.339307  0.339817  …  0.339307  0.338798  0.338289
 0.338787  0.339297  0.339808  0.340319     0.339808  0.339297  0.338787
 0.339285  0.339796  0.340309  0.340821     0.340309  0.339796  0.339285
 ⋮                        